In [1]:
#--Librerias--#
import pandas as pd 
import numpy as np
from sklearn.model_selection import RandomizedSearchCV # búsqueda aleatoria de hiperparámetros 
import joblib  ### para guardar y cargar modelos
from sklearn.preprocessing import StandardScaler ## escalar variables 
from sklearn.feature_selection import SelectKBest, f_classif,chi2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # Regresión logística
from sklearn.ensemble import RandomForestClassifier  # Clasificador bosques aleatoriost 
from sklearn.tree import DecisionTreeClassifier # Arboles de decision 
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler ## escalar variable
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingClassifier #Modelo de clasificacion
from sklearn.metrics import confusion_matrix #### Matriz de confusion 
from sklearn.feature_selection import VarianceThreshold
import seaborn as sns #####Graficos
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

In [2]:
###----Lectura de datos----###
df = pd.read_csv("https://raw.githubusercontent.com/andressj1/Predicci-n-precio-de-m-viles-/refs/heads/main/Datos/train.csv", sep=",")

# Dummies y Escalar variables

In [3]:
# Conversion; categoricas a dummies
df = pd.get_dummies(df, dtype = int)

In [4]:
# Separamos variables explicativas - variable objetivo
X1 = df.drop('price_range',axis=1)
y = df['price_range']

print(X1.shape)
print(y.shape)

(2000, 20)
(2000,)


In [5]:
# Escalar variables 
scaler = StandardScaler()
scaler.fit(X1)
X2 = scaler.transform(X1)
X = pd.DataFrame(X2,columns=X1.columns)

# Selección de variables

*1. Selec From Model*


*1.1. Ridge Classifier*

In [6]:
# Selector de variables con Ridge
sel_ = SelectFromModel(RidgeClassifier(alpha = 0.1, random_state = 24), max_features = 20)
sel_.fit(X, y)
print(sel_.estimator_.coef_)
#Obtener variables seleccionadas
X_new = sel_.get_support()

df_ridge = X.iloc[:,X_new]
df_ridge.info()

[[-1.34532801e-01  4.39342035e-03  1.84588949e-02  7.63127701e-03
  -1.51689882e-02  1.09041706e-02 -4.30302393e-03 -1.60239213e-02
   4.66674797e-03  1.72932783e-02  3.13545188e-03 -1.01191404e-01
  -6.47236127e-02 -6.19800323e-01  1.02174618e-02  9.65697884e-03
  -2.26799343e-02 -1.66633041e-02  2.26085298e-03  8.50563876e-03]
 [-1.59568698e-02  6.81266961e-04 -1.91688052e-02  1.00350708e-02
   6.21744522e-03  8.44138373e-03  7.68520868e-03  3.55753955e-02
   1.67389635e-03 -4.72540068e-02  2.09899412e-03  2.60745139e-02
  -1.17888769e-02 -2.04032075e-01 -2.40711241e-04 -1.68546365e-02
   3.52904526e-02 -6.26281348e-03  1.36577913e-02  1.82149958e-03]
 [-1.21673094e-02 -1.25135013e-02  2.63882976e-03 -1.92633119e-02
   2.49578658e-02 -4.81210136e-02 -3.38357717e-02 -1.73340475e-02
   4.49448528e-02  3.42979989e-02 -1.51689850e-02  6.05470535e-03
  -2.38551460e-02  2.13191629e-01 -3.97760349e-02  7.59412179e-03
  -6.51905143e-03  3.94609304e-02 -2.83849351e-02 -7.71983861e-03]
 [ 1.62

*1.2. Gradient Boosting*

In [7]:
# Selector de variables con Gradient Boosting
sel_ = SelectFromModel(GradientBoostingClassifier(n_estimators = 300, random_state=24))
sel_.fit(X, y)
print(sel_.estimator_.feature_importances_)
#Obtener variables seleccionadas
X_new = sel_.get_support()

df_gb = X.iloc[:,X_new]
df_gb.head()

[1.08877965e-01 2.11928092e-04 4.49781373e-04 1.88058167e-04
 8.64505665e-04 1.34506520e-04 4.07786266e-03 2.27808955e-03
 6.08715463e-03 1.38083830e-03 6.61904816e-04 4.88789195e-02
 2.90736906e-02 7.94075443e-01 6.86893799e-04 1.16873364e-03
 6.37234414e-04 1.59390586e-05 3.17177251e-05 2.18833456e-04]


,battery_power,ram
0,-0.902597,0.391703
1,-0.495139,0.467317
2,-1.537686,0.441498
3,-1.419319,0.594569
4,1.325906,-0.657666


# Prueba de selecciones

Ridge vs RL

In [9]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test,y_train,y_test = train_test_split(df_ridge,y,test_size=0.2, random_state=42)

# Entrenar el modelo
lr = LogisticRegression(max_iter=1000,class_weight="balanced", random_state=42).fit(X_train,y_train)

# Predicciones
y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)
y_pred_prob_train = lr.predict_proba(X_train)
y_pred_prob_test = lr.predict_proba(X_test)

# Matriz de confusión
mc_train = confusion_matrix(y_train, y_pred_train)
mc_test = confusion_matrix(y_test, y_pred_test)

# Métricas para los datos de entrenamiento
precision_train = precision_score(y_train, y_pred_train, average='weighted')
recall_train = recall_score(y_train, y_pred_train, average='weighted')
f1_train = f1_score(y_train, y_pred_train, average='weighted')
accuracy_train = accuracy_score(y_train, y_pred_train)

# Métricas para los datos de prueba
precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')
accuracy_test = accuracy_score(y_test, y_pred_test)

# Imprimir los resultados de entrenamiento
print('-'*30, 'TRAIN', '-'*30)
print(f'Precision (train): {precision_train}')
print(f'Recall (train): {recall_train}')
print(f'F1 score (train): {f1_train}')
print('Train score (accuracy): ', accuracy_train)

# Imprimir los resultados de prueba
print('-'*30, 'TEST', '-'*30)
print(f'Precision (test): {precision_test}')
print(f'Recall (test): {recall_test}')
print(f'F1 score (test): {f1_test}')
print('Test score (accuracy): ', accuracy_test)


------------------------------ TRAIN ------------------------------
Precision (train): 0.9575102644406868
Recall (train): 0.9575
F1 score (train): 0.9574473471419328
Train score (accuracy):  0.9575
------------------------------ TEST ------------------------------
Precision (test): 0.9729502424658674
Recall (test): 0.9725
F1 score (test): 0.972514030860523
Test score (accuracy):  0.9725


Ridge vs Random Forest

In [10]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test,y_train,y_test = train_test_split(df_ridge,y,test_size=0.2, random_state=42)

# Entrenar el modelo
ranfor = RandomForestClassifier(n_estimators=300,
                                  max_depth=30,
                                  n_jobs=-1,
                                  max_leaf_nodes=20,
                                  min_samples_leaf=10,
                                  class_weight="balanced", random_state=42).fit(X_train,y_train)


# Predicciones
y_pred_train = ranfor.predict(X_train)
y_pred_test = ranfor.predict(X_test)
y_pred_prob_train = ranfor.predict_proba(X_train)
y_pred_prob_test = ranfor.predict_proba(X_test)

# Matriz de confusión
mc_train = confusion_matrix(y_train, y_pred_train)
mc_test = confusion_matrix(y_test, y_pred_test)

# Métricas para los datos de entrenamiento
precision_train = precision_score(y_train, y_pred_train, average='weighted')
recall_train = recall_score(y_train, y_pred_train, average='weighted')
f1_train = f1_score(y_train, y_pred_train, average='weighted')
accuracy_train = accuracy_score(y_train, y_pred_train)

# Métricas para los datos de prueba
precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')
accuracy_test = accuracy_score(y_test, y_pred_test)

# Imprimir los resultados de entrenamiento
print('-'*30, 'TRAIN', '-'*30)
print(f'Precision (train): {precision_train}')
print(f'Recall (train): {recall_train}')
print(f'F1 score (train): {f1_train}')
print('Train score (accuracy): ', accuracy_train)

# Imprimir los resultados de prueba
print('-'*30, 'TEST', '-'*30)
print(f'Precision (test): {precision_test}')
print(f'Recall (test): {recall_test}')
print(f'F1 score (test): {f1_test}')
print('Test score (accuracy): ', accuracy_test)


------------------------------ TRAIN ------------------------------
Precision (train): 0.916241865048727
Recall (train): 0.91625
F1 score (train): 0.9161966121372725
Train score (accuracy):  0.91625
------------------------------ TEST ------------------------------
Precision (test): 0.9119007528111303
Recall (test): 0.91
F1 score (test): 0.9104066961831343
Test score (accuracy):  0.91


Gradient Boosting vs RL

In [11]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test,y_train,y_test = train_test_split(df_gb,y,test_size=0.2, random_state=42)

# Entrenar el modelo
lr = LogisticRegression(max_iter=1000,class_weight="balanced", random_state=42).fit(X_train,y_train)

# Predicciones
y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)
y_pred_prob_train = lr.predict_proba(X_train)
y_pred_prob_test = lr.predict_proba(X_test)

# Matriz de confusión
mc_train = confusion_matrix(y_train, y_pred_train)
mc_test = confusion_matrix(y_test, y_pred_test)

# Métricas para los datos de entrenamiento
precision_train = precision_score(y_train, y_pred_train, average='weighted')
recall_train = recall_score(y_train, y_pred_train, average='weighted')
f1_train = f1_score(y_train, y_pred_train, average='weighted')
accuracy_train = accuracy_score(y_train, y_pred_train)

# Métricas para los datos de prueba
precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')
accuracy_test = accuracy_score(y_test, y_pred_test)

# Imprimir los resultados de entrenamiento
print('-'*30, 'TRAIN', '-'*30)
print(f'Precision (train): {precision_train}')
print(f'Recall (train): {recall_train}')
print(f'F1 score (train): {f1_train}')
print('Train score (accuracy): ', accuracy_train)

# Imprimir los resultados de prueba
print('-'*30, 'TEST', '-'*30)
print(f'Precision (test): {precision_test}')
print(f'Recall (test): {recall_test}')
print(f'F1 score (test): {f1_test}')
print('Test score (accuracy): ', accuracy_test)

------------------------------ TRAIN ------------------------------
Precision (train): 0.824604215404147
Recall (train): 0.82625
F1 score (train): 0.8252391302896952
Train score (accuracy):  0.82625
------------------------------ TEST ------------------------------
Precision (test): 0.8198169202347316
Recall (test): 0.8125
F1 score (test): 0.8143234360160251
Test score (accuracy):  0.8125


Gradient Boosting vs Random Forest

In [12]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test,y_train,y_test = train_test_split(df_gb,y,test_size=0.2, random_state=42)

# Entrenar el modelo
ranfor = RandomForestClassifier(n_estimators=300,
                                  max_depth=30,
                                  n_jobs=-1,
                                  max_leaf_nodes=20,
                                  min_samples_leaf=10,
                                  class_weight="balanced", random_state=42).fit(X_train,y_train)


# Predicciones
y_pred_train = ranfor.predict(X_train)
y_pred_test = ranfor.predict(X_test)
y_pred_prob_train = ranfor.predict_proba(X_train)
y_pred_prob_test = ranfor.predict_proba(X_test)

# Matriz de confusión
mc_train = confusion_matrix(y_train, y_pred_train)
mc_test = confusion_matrix(y_test, y_pred_test)

# Métricas para los datos de entrenamiento
precision_train = precision_score(y_train, y_pred_train, average='weighted')
recall_train = recall_score(y_train, y_pred_train, average='weighted')
f1_train = f1_score(y_train, y_pred_train, average='weighted')
accuracy_train = accuracy_score(y_train, y_pred_train)

# Métricas para los datos de prueba
precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')
accuracy_test = accuracy_score(y_test, y_pred_test)

# Imprimir los resultados de entrenamiento
print('-'*30, 'TRAIN', '-'*30)
print(f'Precision (train): {precision_train}')
print(f'Recall (train): {recall_train}')
print(f'F1 score (train): {f1_train}')
print('Train score (accuracy): ', accuracy_train)

# Imprimir los resultados de prueba
print('-'*30, 'TEST', '-'*30)
print(f'Precision (test): {precision_test}')
print(f'Recall (test): {recall_test}')
print(f'F1 score (test): {f1_test}')
print('Test score (accuracy): ', accuracy_test)


------------------------------ TRAIN ------------------------------
Precision (train): 0.8530622164686039
Recall (train): 0.854375
F1 score (train): 0.8533432944686079
Train score (accuracy):  0.854375
------------------------------ TEST ------------------------------
Precision (test): 0.8243541219769343
Recall (test): 0.8175
F1 score (test): 0.8193883197838058
Test score (accuracy):  0.8175


Todas vs Random Forest

In [13]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

# Entrenar el modelo
ranfor = RandomForestClassifier(n_estimators=300,
                                  max_depth=30,
                                  n_jobs=-1,
                                  max_leaf_nodes=20,
                                  min_samples_leaf=10,
                                  class_weight="balanced", random_state=42).fit(X_train,y_train)


# Predicciones
y_pred_train = ranfor.predict(X_train)
y_pred_test = ranfor.predict(X_test)
y_pred_prob_train = ranfor.predict_proba(X_train)
y_pred_prob_test = ranfor.predict_proba(X_test)

# Matriz de confusión
mc_train = confusion_matrix(y_train, y_pred_train)
mc_test = confusion_matrix(y_test, y_pred_test)

# Métricas para los datos de entrenamiento
precision_train = precision_score(y_train, y_pred_train, average='weighted')
recall_train = recall_score(y_train, y_pred_train, average='weighted')
f1_train = f1_score(y_train, y_pred_train, average='weighted')
accuracy_train = accuracy_score(y_train, y_pred_train)

# Métricas para los datos de prueba
precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')
accuracy_test = accuracy_score(y_test, y_pred_test)

# Imprimir los resultados de entrenamiento
print('-'*30, 'TRAIN', '-'*30)
print(f'Precision (train): {precision_train}')
print(f'Recall (train): {recall_train}')
print(f'F1 score (train): {f1_train}')
print('Train score (accuracy): ', accuracy_train)

# Imprimir los resultados de prueba
print('-'*30, 'TEST', '-'*30)
print(f'Precision (test): {precision_test}')
print(f'Recall (test): {recall_test}')
print(f'F1 score (test): {f1_test}')
print('Test score (accuracy): ', accuracy_test)


------------------------------ TRAIN ------------------------------
Precision (train): 0.9125118224597568
Recall (train): 0.913125
F1 score (train): 0.912624542821008
Train score (accuracy):  0.913125
------------------------------ TEST ------------------------------
Precision (test): 0.8723009350549161
Recall (test): 0.8725
F1 score (test): 0.8722376703785446
Test score (accuracy):  0.8725


Todas vs RL

In [14]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

# Entrenar el modelo
lr = LogisticRegression(max_iter=1000,class_weight="balanced", random_state=42).fit(X_train,y_train)

# Predicciones
y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)
y_pred_prob_train = lr.predict_proba(X_train)
y_pred_prob_test = lr.predict_proba(X_test)

# Matriz de confusión
mc_train = confusion_matrix(y_train, y_pred_train)
mc_test = confusion_matrix(y_test, y_pred_test)

# Métricas para los datos de entrenamiento
precision_train = precision_score(y_train, y_pred_train, average='weighted')
recall_train = recall_score(y_train, y_pred_train, average='weighted')
f1_train = f1_score(y_train, y_pred_train, average='weighted')
accuracy_train = accuracy_score(y_train, y_pred_train)

# Métricas para los datos de prueba
precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')
accuracy_test = accuracy_score(y_test, y_pred_test)

# Imprimir los resultados de entrenamiento
print('-'*30, 'TRAIN', '-'*30)
print(f'Precision (train): {precision_train}')
print(f'Recall (train): {recall_train}')
print(f'F1 score (train): {f1_train}')
print('Train score (accuracy): ', accuracy_train)

# Imprimir los resultados de prueba
print('-'*30, 'TEST', '-'*30)
print(f'Precision (test): {precision_test}')
print(f'Recall (test): {recall_test}')
print(f'F1 score (test): {f1_test}')
print('Test score (accuracy): ', accuracy_test)

------------------------------ TRAIN ------------------------------
Precision (train): 0.9787332529774172
Recall (train): 0.97875
F1 score (train): 0.9786865579837116
Train score (accuracy):  0.97875
------------------------------ TEST ------------------------------
Precision (test): 0.9781689842503988
Recall (test): 0.9775
F1 score (test): 0.9774903455943224
Test score (accuracy):  0.9775


Se usara el "Todas vs RL" debido a que muestra un mayor F1 y una mayor precision, ademas no muestra sobreajuste

In [15]:
X['price_range'] = df['price_range']
X.to_csv("Datos\\X", index=False, encoding='utf-8')
